In [1]:
from utils import *
import gensim
from gensim.models import Word2Vec
from tqdm import tqdm_notebook as tqdm

In [ ]:
data_dir = '/datadrive/data_med/'

In [2]:
num_lines = 0
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PR_20190919.tsv', 'r'))
print('PP', num_lines)
num_lines = sum(1 for line in open('/datadrive/MAG_0919/Papers_20190919.tsv', 'r'))
print('PV', num_lines)
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PF_20190919.tsv', 'r'))
print('PF', num_lines)
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PAuAf_20190919.tsv', 'r'))
print('PA', num_lines)

PP 1021237518
PV 89606258
PF 657049405
PA 300853688


In [ ]:
cite_dict = defaultdict(lambda: 0)
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PR_Med_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/PR_Med_20190919.tsv', 'r') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        cite_dict[l[1]] += 1

In [ ]:
pfl = defaultdict(lambda: {})
num_lines = sum(1 for line in open('/datadrive/MAG_0919/Papers_Med_20190919.tsv', 'r'))
res = []
with open('/datadrive/MAG_0919/Papers_Med_20190919.tsv') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        bound = min(3*(2020 - int(l[1])), 60)
        if cite_dict[l[0]] < bound or l[0] == '' or l[1] == '' or l[2] == '' or l[3] == '' and l[4] == '' or int(l[1]) < 1900:
            continue
        pi = {'id': l[0], 'title': l[2], 'type': 'paper', 'time': int(l[1])}
        pfl[l[0]] = pi
        res += [[int(l[1]), cite_dict[l[0]]]]

In [ ]:
from pytorch_transformers import *
device = torch.device("cuda:1")
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetModel.from_pretrained('xlnet-base-cased',
                                    output_hidden_states=True,
                                    output_attentions=True).to(device)

In [ ]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PAb_Med_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/PAb_Med_20190919.tsv') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        try:
            l = l.split('\t')
            if l[0] in pfl:
                input_ids = torch.tensor([tokenizer.encode(pfl[l[0]]['title'])]).to(device)[:, :64]
                if len(input_ids[0]) < 4:
                    continue
                all_hidden_states, all_attentions = model(input_ids)[-2:]
                rep = (all_hidden_states[-2][0] * all_attentions[-2][0].mean(dim=0).mean(dim=0).view(-1, 1)).sum(dim=0)
                pfl[l[0]]['emb'] = rep.tolist()
        except Exception as e:
            print(e)

In [ ]:
vfi_ids = {}
num_lines = sum(1 for line in open('/datadrive/MAG_0919/vfi_vector.tsv', 'r'))
with open('/datadrive/MAG_0919/vfi_vector.tsv', 'r') as fin:
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        vfi_ids[l[0]] = True

In [ ]:
graph = Graph()
rem = []
num_lines = sum(1 for line in open('/datadrive/MAG_0919/Papers_Med_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/Papers_Med_20190919.tsv') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        if l[0] not in pfl or l[4] != 'en' or 'emb' not in pfl[l[0]] or l[3] not in vfi_ids:
            continue
        rem += [l[0]]
        vi = {'id': l[3], 'type': 'venue', 'attr': l[-2]}
        graph.add_edge(pfl[l[0]], vi, time = int(l[1]), relation_type = 'PV_' + l[-2])
pfl = {i: pfl[i] for i in rem}
print(len(pfl))

In [ ]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PR_Med_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/PR_Med_20190919.tsv', 'r') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        if l[0] in pfl and l[1] in pfl:
            p1 = pfl[l[0]]
            p2 = pfl[l[1]]
            if p1['time'] >= p2['time']:
                graph.add_edge(p1, p2, time = p1['time'], relation_type = 'PP_cite')

In [ ]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PF_Med_20190919.tsv', 'r'))
ffl = {}
with open('/datadrive/MAG_0919/PF_Med_20190919.tsv', 'r') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        if l[0] in pfl and l[1] in vfi_ids:
            ffl[l[1]] = True

In [ ]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/FHierarchy_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/FHierarchy_20190919.tsv', 'r') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        if l[0] in ffl and l[1] in ffl:
            fi = {'id': l[0], 'type': 'field', 'attr': l[2]}
            fj = {'id': l[1], 'type': 'field', 'attr': l[3]}
            graph.add_edge(fi, fj, relation_type = 'FF_in')
            ffl[l[0]] = fi
            ffl[l[1]] = fj

In [ ]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PF_Med_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/PF_Med_20190919.tsv', 'r') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        if l[0] in pfl and l[1] in ffl and type(ffl[l[1]]) == dict:
            pi = pfl[l[0]]
            fi = ffl[l[1]]
            graph.add_edge(pi, fi, time = pi['time'], relation_type = 'PF_in_' + fi['attr'])

In [ ]:
coi = defaultdict(lambda: [])
coa = defaultdict(lambda: {})
num_lines = sum(1 for line in open('/datadrive/MAG_0919/PAuAf_Med_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/PAuAf_Med_20190919.tsv', 'r') as fin:
    fin.readline()
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        if l[0] in pfl and l[2] in vfi_ids:
            pi = pfl[l[0]]
            ai = {'id': l[1], 'type': 'author'}
            fi = {'id': l[2], 'type': 'affiliation'}
            coa[l[0]][int(l[-1])] = ai
            coi[l[0]] += [fi]
            graph.add_edge(ai, fi, relation_type = 'in')
            pid = graph.node_forward['paper'][l[0]]
            for rel in graph.edge_list['paper']['venue']:
                for vid in graph.edge_list['paper']['venue'][rel][pid]:
                    graph.add_edge(ai, graph.node_bacward['venue'][vid], relation_type = 'APV_in')
            for rel in graph.edge_list['paper']['field']:
                for fid in graph.edge_list['paper']['field'][rel][pid]:
                    graph.add_edge(ai, graph.node_bacward['field'][fid], relation_type = 'APF_in')

In [ ]:
for pid in tqdm(coa):
    pi = pfl[pid]
    max_seq = max(coa[pid].keys())
    for seq_i in coa[pid]:
        ai = coa[pid][seq_i]
        if seq_i == 1:
            graph.add_edge(ai, pi, time = pi['time'], relation_type = 'AP_write_first')
        elif seq_i == max_seq:
            graph.add_edge(ai, pi, time = pi['time'], relation_type = 'AP_write_last')
        else:
            graph.add_edge(ai, pi, time = pi['time'], relation_type = 'AP_write_other')
        for seq_j in coa[pid]:
            if seq_j > seq_i:
                aj = coa[pid][seq_j]
                graph.add_edge(ai, aj, time = pi['time'], relation_type = 'APA_coauthor', directed = False)

In [ ]:
for pid in tqdm(coi):
    pi = pfl[pid]
    for seq_i, fi in enumerate(coi[pid]):
        for seq_j, fj in enumerate(coi[pid]):
            if seq_j > seq_i:
                graph.add_edge(fi, fj, time = pi['time'], relation_type = 'IPI_coauthor', directed = False)

In [ ]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/vfi_vector.tsv', 'r'))
with open('/datadrive/MAG_0919/vfi_vector.tsv', 'r') as fin:
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        ser = l[0]
        for idx in ['venue', 'field', 'affiliation']:
            if ser in graph.node_forward[idx]:
                graph.node_bacward[idx][graph.node_forward[idx][ser]]['node_emb'] = np.array(l[1].split(' '))

In [ ]:
for idx in ['venue', 'field', 'affiliation']:
    tot = 0
    for i in graph.node_bacward[idx]:
        if 'node_emb' in i:
            tot += 1
    print(tot, len(graph.node_bacward[idx]))

In [ ]:
for target_type in graph.node_bacward:
    print(target_type, len(graph.node_bacward[target_type]))

In [ ]:
for target_type in graph.edge_list:
    print(target_type)
    for source_type in graph.edge_list[target_type]:
        print('-' * 4 + source_type)
        for rel_type in graph.edge_list[target_type][source_type]:
            tot = 0
            for tid in graph.edge_list[target_type][source_type][rel_type]:
                tot += len(graph.edge_list[target_type][source_type][rel_type][tid])
            print('-' * 8 + rel_type, tot)

In [ ]:
num_lines = sum(1 for line in open('/datadrive/MAG_0919/SeqName_Med_20190919.tsv', 'r'))
with open('/datadrive/MAG_0919/SeqName_Med_20190919.tsv', 'r') as fin:
    for l in tqdm(fin, total = num_lines):
        l = l[:-1].split('\t')
        key = l[2]
        if key in ['conference', 'journal', 'repository', 'patent']:
            key = 'venue'
        if key == 'fos':
            key = 'field'
        if l[0] in graph.node_forward[key]:
            s = graph.node_bacward[key][graph.node_forward[key][l[0]]]
            s['name'] = l[1]

In [ ]:
res = []
for idx, pi in enumerate(graph.node_bacward['paper']):
    pi['citation'] = len(graph.edge_list['paper']['paper']['PP_cite'][idx])
    res += [[pi['title'], pi['citation']]]
res.sort(key=lambda x: x[1], reverse = True)
res[:10]

In [ ]:
res = []
for idx, ai in enumerate(graph.node_bacward['author']):
    citation = 0
    for rel in graph.edge_list['author']['paper'].keys():
        for pid in graph.edge_list['author']['paper'][rel][idx]:
            citation += graph.node_bacward['paper'][pid]['citation']
    ai['citation'] = citation
    res += [[ai['name'], ai['citation']]]
res.sort(key=lambda x: x[1], reverse = True)
res[:10]

In [ ]:
res = []
for idx, fi in enumerate(graph.node_bacward['affiliation']):
    citation = 0
    for aid in graph.edge_list['affiliation']['author']['in'][idx]:
        citation += graph.node_bacward['author'][aid]['citation']
    fi['citation'] = citation
    res += [[fi['name'], fi['citation']]]
res.sort(key=lambda x: x[1], reverse = True)
res[:10]

In [ ]:
res = []
for idx, vi in enumerate(graph.node_bacward['venue']):
    citation = 0
    for rel in graph.edge_list['venue']['paper'].keys():
        for pid in graph.edge_list['venue']['paper'][rel][idx]:
            citation += graph.node_bacward['paper'][pid]['citation']
    vi['citation'] = citation
    res += [[vi['name'], vi['citation'], vi['attr']]]
res.sort(key=lambda x: x[1], reverse = True)
res[:10]

In [ ]:
res = []
for idx, fi in enumerate(graph.node_bacward['field']):
    citation = 0
    for rel in graph.edge_list['field']['paper'].keys():
        for pid in graph.edge_list['field']['paper'][rel][idx]:
            citation += graph.node_bacward['paper'][pid]['citation']
    fi['citation'] = citation
    res += [[fi['name'], fi['citation'], fi['attr']]]
res.sort(key=lambda x: x[1], reverse = True)
res[:100]

In [ ]:
graph.propagate_feature()

In [ ]:
del graph.node_bacward

In [ ]:
dill.dump(graph, open(data_dir + 'graph.pk', 'wb'))